### Initial exploration of data features
In this notebook we will analyse some of the initial data, by finding out what features are available, what granularity the data comes in, as well as seeing if there is any obvious noise we need to take into account.

In [ ]:
experiment = "data/experiments/experiment_2/"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
## Loading in the data from data/experiments/experiment_1, which contains button_presses.csv for the labels, and other csv files for the features. In experiments_1/meta we have the system time and the device info
def load_data(path, within_range=True, temp_features=True):
    # Load the starting time
    time_df = pd.read_csv(path + 'meta/time.csv')
    start_time = time_df.loc[time_df['event'] == 'START', 'system time'].iloc[0]

    data_frames = []
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            df = pd.read_csv(path + filename)
            # Check if 'Time (s)' column exists
            if 'Time (s)' in df.columns:
                # Convert 'Time (s)' column to datetime index for each dataframe
                df.index = pd.to_datetime(df['Time (s)'], unit='s', origin=pd.Timestamp(start_time, unit='s'))
                data_frames.append(df)
            else:
                print(f"'Time (s)' column not found in file: {filename}")
                print(f"Columns found: {df.columns}")
    
    # Concatenate dataframes
    data = pd.concat(data_frames)
    
    # resample to 10 Hz
    data_resampled = data.resample('100ms').mean()
    
    # Load label dataset
    labels = pd.read_csv(path+'button_presses.csv', names=['Timestamp', 'Label'])
    labels['Timestamp'] = pd.to_datetime(labels['Timestamp'], unit='s')
    
    # Filter timestamps within label range
    if within_range:
        first_label_timestamp = labels['Timestamp'].iloc[0]
        last_label_timestamp = labels['Timestamp'].iloc[-1]
        data_resampled = data_resampled[(data_resampled.index >= first_label_timestamp) & (data_resampled.index <= last_label_timestamp)]
    
    if len(data_resampled):
        # Add labels
        def get_recent_label(row):
            return labels[labels['Timestamp'] <= row.name]['Label'].iloc[-1]

        data_resampled['Label'] = data_resampled.apply(get_recent_label, axis=1)

        # Add temporal label features
        def get_time_until_next(row):
            next_label = labels[labels['Timestamp'] > row.name]['Timestamp'].min()
            if pd.isnull(next_label):
                return pd.NaT
            else:
                return (next_label - row.name).total_seconds()

        def get_time_since_previous(row):
            previous_label = labels[labels['Timestamp'] < row.name]['Timestamp'].max()
            if pd.isnull(previous_label):
                return pd.NaT
            else:
                return (row.name - previous_label).total_seconds()

        data_resampled['Time_Until_Next_Label'] = data_resampled.apply(get_time_until_next, axis=1)
        data_resampled['Time_Since_Previous_Label'] = data_resampled.apply(get_time_since_previous, axis=1)
        data_resampled['Time_Until_Next_Label'] = data_resampled['Time_Until_Next_Label'].fillna(0.0)
        data_resampled['Time_Since_Previous_Label'] = data_resampled['Time_Since_Previous_Label'].fillna(0.0)
    return data_resampled

data = load_data(experiment)
# display(data)

In [ ]:
import xml.etree.ElementTree as ET
from datetime import datetime

# Define a conversion function
def convert_timestamp(timestamp):
    datetime_obj = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S.%fZ')
    return datetime_obj

# Load the xml file into a dataframe
def load_xml(path, convert_time=True):
    # Parse the XML file
    tree = ET.parse(path + 'activity_11340269258.tcx')
    root = tree.getroot()

    # Define the namespaces
    namespaces = {
        'tc': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
        'activity': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
        'ns3': 'http://www.garmin.com/xmlschemas/ActivityExtension/v2',
        'ns5': 'http://www.garmin.com/xmlschemas/ActivityGoals/v1',
        'ns2': 'http://www.garmin.com/xmlschemas/UserProfile/v2',
        'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
        'ns4': 'http://www.garmin.com/xmlschemas/ProfileExtension/v1'
    }

    # Extract data from XML and create a dictionary
    xml_data = {'Time': [], 'AltitudeMeters': [], 'HeartRate': []}

    for trackpoint in root.findall('.//tc:Trackpoint', namespaces):
        time = trackpoint.find('tc:Time', namespaces).text
        altitude = trackpoint.find('tc:AltitudeMeters', namespaces).text
        heart_rate = trackpoint.find('tc:HeartRateBpm/tc:Value', namespaces).text

        xml_data['Time'].append(time)
        xml_data['AltitudeMeters'].append(altitude)
        xml_data['HeartRate'].append(heart_rate)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(xml_data)
    
    df['AltitudeMeters'] = df['AltitudeMeters'].astype(float)
    df['HeartRate'] = df['HeartRate'].astype(float)
    
    # Apply the conversion function to the 'Time' column
    if convert_time:
        df['Time'] = df['Time'].apply(convert_timestamp)
    
    df = df.set_index('Time')
    
    return df

xml_data = load_xml(experiment)

In [ ]:
# Merges the csv and xml data
def merge(data, xml_data):
    first_timestamp = data.index[0]
    last_timestamp = data.index[-1]
    df_filtered = xml_data[(xml_data.index >= first_timestamp) & (xml_data.index <= last_timestamp)]
    merged_df = pd.merge(data, df_filtered, left_index=True, right_index=True, how='left')
    
    return merged_df
df = merge(data, xml_data)


## Load dataframe from file

In [ ]:
df = pd.read_csv(experiment + 'merged/added_features.csv')

## Change labels to numbers

In [ ]:
df['Label'] = df['Label'].astype('category').cat.codes

## Remove unuseful columns

In [ ]:
def remove_columns(df, columns):
    return df.drop(columns=columns)

# 'X (hPa)' nog evt
columns = ['Velocity (m/s)', 'Direction (°)', 'Distance (cm)', 'Horizontal Accuracy (m)', 'Time_Until_Next_Label', 'Time_Since_Previous_Label', 'Time (s)', 'Latitude (°)', 'Longitude (°)']
df_filt = df.drop(columns=columns)
df_filt = df_filt.rename(columns={'Time (s).1': 'Time (s)'})

In [ ]:
df_filt

## Scaling values between 0 and 1

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def scale_values(df):
    exclude_columns = ['Time (s)', 'Label']  # Add a comma between the column names
    scaler = MinMaxScaler()
    scaler.fit(df.loc[:, ~df.columns.isin(exclude_columns)])
    scaled_data = scaler.transform(df.loc[:, ~df.columns.isin(exclude_columns)])
    df.loc[:, ~df.columns.isin(exclude_columns)] = scaled_data
    return df

scaled_df = scale_values(df_filt)


In [ ]:
scaled_df

## Interpolation

In [ ]:
from ImputationMissingValues import ImputationMissingValues

# Interpolates specified columns of the dataframe (all columns with NaNs as default)
def interpolate(df, columns=None):
    if columns is None:
        columns = df.columns[df.isna().any()].tolist()
    
    imputer = ImputationMissingValues()
    for column in columns:
        df = imputer.impute_interpolate(df, column)
    
    return df

interpolated_df = interpolate(scaled_df)
# interpolated_df

In [ ]:
interpolated_df

## Adding features
Mean, std, min, max, difference

In [ ]:
def create_features(df, seconds, columns=None):
    # Create a new DataFrame to store the aggregated values
    new_df = pd.DataFrame()

    # Define the window size
    window_size = seconds * 10
    
    if columns == None:
        columns = df.loc[:, ~df.columns.str.startswith('Label')].columns

    # Iterate over the rolling windows in the original DataFrame
    for i in range(len(df) - window_size + 1):
        # Select a rolling window subset
        subset = df.iloc[i:i+window_size]

        # Iterate over each column in the subset
        for col in columns:
            if col == "Time (s)":
                new_df.loc[i, f'Starttime (s)'] = subset[col].iloc[0]
                new_df.loc[i, f'Endtime (s)'] = subset[col].iloc[-1]
            
            else:
                col_mean = subset[col].mean()
                col_std = subset[col].std()
                col_min = subset[col].min()
                col_max = subset[col].max()
                col_diff = subset[col].iloc[-1] - subset[col].iloc[0]

                # Create new columns in the new DataFrame
                new_df.loc[i, f'{col} mean'] = col_mean
                new_df.loc[i, f'{col} std'] = col_std
                new_df.loc[i, f'{col} min'] = col_min
                new_df.loc[i, f'{col} max'] = col_max
                new_df.loc[i, f'{col} diff'] = col_diff

        # Get the most frequent label within the window
        most_frequent_label = subset['Label'].mode().iloc[0]
        new_df.loc[i, 'Label'] = most_frequent_label

    # Reset the index of the new DataFrame
    new_df.reset_index(drop=True, inplace=True)

    # Print the new DataFrame
    return new_df


In [ ]:
# df_full = create_features(interpolated_df, 5, columns=None)

## Robbie vanaf hier runne!! ga nu renne

In [ ]:
df_full = pd.read_csv(experiment + 'merged/added_features_stats_scaled.csv')

In [ ]:
df_full

## Increase samples of minority classes

In [ ]:
features = df_full.drop(['Label'], axis=1)
labels = df_full['Label']

In [ ]:
encoded_labels = df_full['Label'].astype('category').cat.codes

# Create a dictionary to store the mapping of encoded labels to original words
label_mapping = dict(zip(encoded_labels, df_full['Label']))

In [ ]:
class_labels = ['Break', 'Falling', 'Lowering', 'Overhanging', 'Slab', 'Straight']
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(6,5))
label_counts = np.bincount(df_full['Label'].astype('category').cat.codes)
x = np.arange(len(class_labels))
plt.bar(x, label_counts, edgecolor='black')
plt.xticks(range(len(class_labels)), class_labels)
plt.xticks(rotation=45, ha='right')
plt.ylabel('Count')
plt.title('Distribution of Encoded Labels')
plt.tight_layout()
plt.savefig("labels_original.pdf")
plt.show()

In [ ]:
label_counts = labels.value_counts()

# Calculate the ratios between labels
label_ratios = label_counts / label_counts.sum()

# Print the label ratios
print(label_counts)

In [ ]:
# The second most frequent label must become 60% of the most frequent label
# All other labels scale accordingly
minority_ratio = 0.6 * max(label_counts)/sorted(label_counts)[-2]

# Define custom resampling ratios for each label
resampling_ratios = {
    0: label_counts[0],  # Resampling ratio for label 0
    1: int(minority_ratio * label_counts[1]),   # Resampling ratio for label 1
    2: int(minority_ratio * label_counts[2]),   # Resampling ratio for label 2
    3: int(minority_ratio * label_counts[3]),   # Resampling ratio for label 3
    4: int(minority_ratio * label_counts[4]),   # Resampling ratio for label 4
    5: int(minority_ratio * label_counts[5])    # Resampling ratio for label 5
}

In [ ]:
from imblearn.over_sampling import SMOTE
# Create an instance of SMOTE
smote = SMOTE(sampling_strategy=resampling_ratios)

# Resample the data using SMOTE
features_resampled, labels_resampled = smote.fit_resample(features, labels)

In [ ]:
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(6,5))
label_counts = np.bincount(labels_resampled)
x = np.arange(len(class_labels))
plt.bar(x, label_counts, edgecolor='black')
plt.xticks(range(len(class_labels)), class_labels)
plt.xticks(rotation=45, ha='right')
plt.ylabel('Count')
plt.title('Distribution of Encoded Labels')
plt.tight_layout()
plt.savefig("labels_sampled.pdf")
plt.show()

## Principal Component Analysis

In [ ]:
from DataTransformation import PrincipalComponentAnalysis

# PCA function where number of PC's and columns used can be specified (all columns but the label as default).
# Adds new PCA columns to the dataframe
def pca(df, num_comp, columns=None):
    if columns is None:
        columns = df.columns
    PCA = PrincipalComponentAnalysis()
    pca_df = PCA.apply_pca(df, columns, num_comp)
    
    return pca_df

def determine_pc_explained_variance(df, columns = None):
    if columns is None:
        columns = df.columns
    PCA = PrincipalComponentAnalysis()
    ratio, feature_importances = PCA.determine_pc_explained_variance(df, columns)
    
    return ratio, feature_importances

ratio, feature_importances = determine_pc_explained_variance(features_resampled, columns = None)

In [ ]:
# n_features = len(feature_importances)
# n_features = 25

# feature_names = features_resampled.columns  # Provide the names of your original features

# sorted_indices = sorted(range(len(feature_importances)), key=lambda k: feature_importances[k], reverse=True)
# sorted_feature_importances = [feature_importances[i] for i in sorted_indices]
# sorted_feature_names = [feature_names[i] for i in sorted_indices]

# plt.figure(figsize=(8, 6))
# plt.bar(range(n_features), sorted_feature_importances[:25], tick_label=sorted_feature_names[:25])
# plt.xticks(rotation=90, fontsize=9)
# plt.xlabel('Features')
# plt.ylabel('Importance')
# plt.title('Feature Importances with PCA')
# plt.show()

In [ ]:
cumulative_variance = np.cumsum(ratio)
n_components = np.argmax(cumulative_variance >= 0.95) + 1

print("Number of components to keep:", n_components)

In [ ]:
pca_df = pca(features_resampled, n_components).copy()
pca_df

## Support Vector Machine

In [ ]:
from LearningAlgorithms import ClassificationAlgorithms
from sklearn.utils import shuffle


X = pca_df.iloc[:, -n_components:]
Y = labels_resampled

one = int(0.10 * len(X))
two = int(0.30 * len(X))

X_train = pd.concat([X[:one], X[two:]], axis=0)
X_test = X[one:two]
Y_train = pd.concat([Y[:one], Y[two:]], axis=0)
Y_test = Y[one:two]

In [ ]:
# clf = ClassificationAlgorithms()
# pred_training_y, pred_test_y, frame_prob_training_y, frame_prob_test_y = clf.support_vector_machine_with_kernel(X_train, Y_train, X_test, C=100,  kernel='rbf', gamma=0.0001, gridsearch=False, print_model_details=False)

## Evaluation

In [ ]:
import pickle
with open(experiment + 'svm/pred_test_y3', 'rb') as file:
    pred_test_y = pickle.load(file)

### Check label distribution in train/test set

In [ ]:
plt.hist(Y_train, edgecolor='black')
plt.xlabel('Encoded Labels')
plt.ylabel('Count')
plt.title('Distribution of Encoded Labels')
plt.show()

In [ ]:
plt.hist(Y_test, edgecolor='black')
plt.xlabel('Encoded Labels')
plt.ylabel('Count')
plt.title('Distribution of Encoded Labels')
plt.show()

### Report evaluation metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

def metrics(labels, predictions):
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-Score:", f1)
    
    cm = confusion_matrix(labels, predictions)
    classes = np.arange(6)
    
    sns.set_style("whitegrid")
    sns.set_context("paper")
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.5)
    sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.savefig("confusion_matrix.pdf", bbox_inches='tight')
    plt.show()
    
    return accuracy, precision, recall, f1

metrics(Y_test, pred_test_y)

## Plotting

In [ ]:
# import seaborn as sns
# from scipy import stats
# import matplotlib.pyplot as plt
# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()
# from statsmodels.tsa.seasonal import seasonal_decompose

# def plot_data(data, output_path):
#     # Set Seaborn style and context
#     sns.set_style("whitegrid")
#     sns.set_context("paper")

#     # Line plot
#     fig, ax = plt.subplots()
#     sns.lineplot(data=data, palette="tab10", linewidth=1.5, ax=ax)
#     ax.set_title('Time Series Line Plot')
#     ax.set_xlabel('Time')
#     ax.set_ylabel('Values')
#     fig.savefig(output_path + 'line_plot.pdf')

#     # Distribution of the data
#     fig, ax = plt.subplots()
#     sns.kdeplot(data=data, ax=ax, fill=True)
#     ax.set_title('Data Distribution Plot')
#     ax.set_xlabel('Values')
#     fig.savefig(output_path + 'distribution_plot.pdf')

#     # Boxplots of the data
#     fig, ax = plt.subplots()
#     sns.boxplot(data=data, palette="tab10", ax=ax)
#     ax.set_title('Data Boxplot')
#     fig.savefig(output_path + 'boxplot.pdf')

#     # Correlation matrix of the data
#     fig, ax = plt.subplots()
#     sns.heatmap(data.corr(), annot=True, cmap='coolwarm', ax=ax)
#     ax.set_title('Data Correlation Matrix')
#     fig.savefig(output_path + 'correlation_matrix.pdf')

#     # Pairwise relationships
#     fig, ax = plt.subplots()
#     sns.pairplot(data)
#     ax.set_title('Pairwise Relationships')
#     fig.savefig(output_path + 'pairwise_relationships.pdf')

#     # Histogram
#     fig, ax = plt.subplots()
#     data.hist(bins=30, ax=ax)
#     ax.set_title('Data Histogram')
#     fig.savefig(output_path + 'histogram.pdf')

#     # Time series decomposition
#     for col in data.columns:
#         try:
#             result = seasonal_decompose(data[col], model='additive', period=1)
#             fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1, figsize=(10,8))
#             result.observed.plot(ax=ax1)
#             ax1.set_ylabel('Observed')
#             result.trend.plot(ax=ax2)
#             ax2.set_ylabel('Trend')
#             result.seasonal.plot(ax=ax3)
#             ax3.set_ylabel('Seasonal')
#             result.resid.plot(ax=ax4)
#             ax4.set_ylabel('Residual')
#             fig.savefig(output_path + f'{col}_decomposition.pdf')
#         except:
#             print(f"Cannot decompose {col}")

# output_path = experiment + "figures/"
# plot_data(df, output_path)